In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score


In [3]:
# Load your CSVs
train_df = pd.read_csv("train (1).csv")
test_df = pd.read_csv("test (1).csv")
dev_df = pd.read_csv("dev.csv")

# Peek data
print(train_df.head())
print(train_df.columns)

              pid                                               text  labels
0  train_pid_7991  At this point just genuinely curious what sort...       0
1  train_pid_7992  I have literally tried everything.... : I'm st...       0
2  train_pid_7995  I'm really struggling : So I don't know how to...       0
3  train_pid_7996  My meds aren’t making my depression any better...       0
4  train_pid_7997  Hi I'm unwell : I'm 21 now, "vice ridden", and...       0
Index(['pid', 'text', 'labels'], dtype='object')


In [6]:
train_df["labels"].value_counts()

labels
1    3101
2    2255
0     650
Name: count, dtype: int64

In [11]:
# Show 3 random samples per label
for label in train_df["labels"].unique():
    print(f"\nLabel {label} samples:\n")
    samples = train_df[train_df["labels"] == label].sample(3, random_state=42)["text"]
    for i, text in enumerate(samples, 1):
        print(f"{i}. {text}")




Label 0 samples:

1. Having a break down and wanted to hurt myself : For some reason today im having a anxiety panic attack and suddenly i just wanted to cry. I wanted to talk to someone so bad but im to scared that people think im annoying! I hide in my bathroom and cry nonstop for 2 dam hours straight and luckily i didn't bring any razor so i won't cut myself.... My relationship are toxic! It so toxic that i never think something good about myself and everyone around make me believe that. In reality i just need to stand up for myself - im a pathetic loser that to scare to committed to things, and i won't change it because im a coward ! Ha i sound so mess up and stupid dont i :)
2. Not sure who else needs this, but you've made it through another day of existing. : I know how hard it is to be here sometimes, especially when the medicines that you're supposed to use aren't helping as much as you'd like them to, or the people around you don't seem to know how much you're struggling inte

In [3]:
print(train_df.columns)


Index(['pid', 'text', 'labels'], dtype='object')


In [4]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Encode labels
encoder = LabelEncoder()
train_df['labels'] = encoder.fit_transform(train_df['labels'])
test_df['labels'] = encoder.transform(test_df['labels'])
dev_df['labels'] = encoder.transform(dev_df['labels'])

# Tokenize text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['text'])

X_train = tokenizer.texts_to_sequences(train_df['text'])
X_test = tokenizer.texts_to_sequences(test_df['text'])
X_dev = tokenizer.texts_to_sequences(dev_df['text'])

# Pad sequences to fixed length
max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len, padding="post")
X_test = pad_sequences(X_test, maxlen=max_len, padding="post")
X_dev = pad_sequences(X_dev, maxlen=max_len, padding="post")

y_train = train_df['labels'].values
y_test = test_df['labels'].values
y_dev = dev_df['labels'].values

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

vocab_size = 10000
embedding_dim = 128
max_len = 100
num_classes = len(encoder.classes_)

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    LSTM(128, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Build the model explicitly
model.build(input_shape=(None, max_len))
model.summary()

optimizer = SGD(learning_rate=0.01, momentum=0.9)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',   # for integer-encoded labels
    optimizer=optimizer,
)  
# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=20,
    batch_size=32
)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,420,035 (5.42 MB)

 Trainable params: 1,420,035 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.9456 - val_loss: 0.8481
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.9209 - val_loss: 0.7961
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.9201 - val_loss: 0.8109
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 0.9184 - val_loss: 0.8070
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.9170 - val_loss: 0.7973
Epoch 6/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.9188 - val_loss: 0.8257
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.9163 - val_loss: 0.8336
Epoch 8/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.9152 - val_loss: 0.8015
Epoch 9/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.9167 - val_loss: 0.8225
Epoch 10/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.9143 - val_loss: 0.8034
Epoch 11/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.9127 - val_loss: 0.7841
Epoch 12/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 

In [17]:
target_names = encoder.classes_.astype(str)  # ensures they are strings
print(classification_report(y_test, y_pred_classes, target_names=target_names))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       228
           1       0.69      0.83      0.76      2169
           2       0.41      0.30      0.35       848

    accuracy                           0.64      3245
   macro avg       0.37      0.38      0.37      3245
weighted avg       0.57      0.64      0.60      3245



c:\Users\Pc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Pc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Pc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
# Convert classes to string labels
target_names = [str(label) for label in encoder.classes_]

print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes, target_names=target_names))


Accuracy: 0.6357473035439137
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       228
           1       0.69      0.83      0.76      2169
           2       0.41      0.30      0.35       848

    accuracy                           0.64      3245
   macro avg       0.37      0.38      0.37      3245
weighted avg       0.57      0.64      0.60      3245



c:\Users\Pc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Pc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Pc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Report
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
#print(classification_report(y_test, y_pred_classes, target_names=encoder.classes_))


102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Accuracy: 0.6357473035439137


In [34]:
import pandas as pd

# Load test data
test_df = pd.read_csv("train (1).csv")

# # Take one sample (first row)
# sample_text = test_df['text'].iloc[5]  
# print("Sample text:", sample_text)

sample_text="i am depressed"

In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you already have the same tokenizer used in training
maxlen=10
seq = tokenizer.texts_to_sequences([sample_text])
padded = pad_sequences(seq, maxlen=maxlen)  # same maxlen as training


In [36]:
import numpy as np

# Predict
pred = model.predict(padded)

# Convert to class
pred_class = np.argmax(pred, axis=1)
print("Predicted class:", pred_class[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted class: 1


# naive bayes

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(train_df['text'])
X_test_tfidf = vectorizer.transform(test_df['text'])
X_dev_tfidf = vectorizer.transform(dev_df['text'])

# Naive Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred_test = nb_model.predict(X_test_tfidf)
y_pred_dev = nb_model.predict(X_dev_tfidf)

# Evaluation
print("Test Accuracy:", accuracy_score(y_test, y_pred_test))
print("Dev Accuracy:", accuracy_score(y_dev, y_pred_dev))
# print("\nClassification Report:\n", classification_report(y_test, y_pred_test, target_names=encoder.classes_))


Test Accuracy: 0.6776579352850539
Dev Accuracy: 0.542


In [42]:
# Sample text to predict
sample_text = ["I feel hopeless and nothing seems to bring me joy anymore"]  # replace with your text

# Transform using the same TF-IDF vectorizer
sample_tfidf = vectorizer.transform(sample_text)

# Predict
sample_pred = nb_model.predict(sample_tfidf)
predicted_label = encoder.inverse_transform(sample_pred)

print("Predicted label:", predicted_label[0])


Predicted label: 1


In [43]:
import pickle

# Save the model
with open("nb_model.pkl", "wb") as model_file:
    pickle.dump(nb_model, model_file)

# Save the vectorizer
with open("vectorizer.pkl", "wb") as vec_file:
    pickle.dump(vectorizer, vec_file)

print("✅ Model and vectorizer saved successfully!")


✅ Model and vectorizer saved successfully!


In [44]:
from sklearn.preprocessing import LabelEncoder
import pickle

# Example: fitting encoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_df['labels'])
y_test = encoder.transform(test_df['labels'])
y_dev = encoder.transform(dev_df['labels'])

# --- Save encoder ---
with open("label_encoder.pkl", "wb") as enc_file:
    pickle.dump(encoder, enc_file)

print("✅ Label encoder saved successfully!")


✅ Label encoder saved successfully!
